In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# The Evidence Weighted Majority with Normalization Algorithm (EWMWN)
The EWMWN class implements a learning algorithm with multiple SGDClassifier experts. Initially, it assigns equal mass to each expert. It makes aggregated predictions based on the experts' outputs. If the coarsened prediction results in an error, the masses of incorrect experts are reduced by a factor of beta  and then all the experts' masses are normalized to maintain the sum of the masses at 1. No updates are made when all experts agree, as the situation is uninformative. The goal is to aggregate predictions in adversarial settings and minimize errors through dynamic mass updates.

In [2]:
class EWMWN:
    def __init__(self, num_experts, beta):
        self.total_errors = 0
        self.num_experts = num_experts
        self.beta = beta
        self.experts = list(range(self.num_experts))
        self.experts_info = {}  # Dictionary to store expert information

    def initialize_experts(self, X_train, y_train):
        """
        Initializes experts as SGDClassifier models.
        Each expert is trained on a subset of the training data.
        """

        for i in range(self.num_experts):
            X_subtrain, _, y_subtrain, _ = train_test_split(X_train, y_train, test_size=0.5)
            clf = SGDClassifier(max_iter=1000, tol=1e-3)
            clf.fit(X_subtrain, y_subtrain)

            # Store model information, initial mass, and empty predictions list for each expert
            self.experts_info[i] = {
                'model': clf,
                'mass': 1.0 / self.num_experts,
                'predictions': []
            }

        print(f"Initialized {self.num_experts} experts: {self.experts_info}")

    def coarsening(self, t):
        """
        Aggregates the predictions of active experts and calculates the coarsened prediction.
        Returns the coarsened masses for predictions of 1 and 0.
        """

        # Identify experts predicting 1 and 0 at time t
        experts_predicted_1 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 1}
        experts_predicted_0 = {i for i, info in self.experts_info.items() if info['predictions'][t] == 0}

        print(f"Experts predicting 1: {experts_predicted_1}")
        print(f"Experts predicting 0: {experts_predicted_0}")

        # Compute the coarsened mass function
        mass_1 = np.sum([info['mass'] for i, info in self.experts_info.items() if info['mass'] > 0 and info['predictions'][t] == 1])
        mass_0 = np.sum([info['mass'] for i, info in self.experts_info.items() if info['mass'] > 0 and info['predictions'][t] == 0])

        print(f"Mass_1: {mass_1}, Mass_0: {mass_0}")

        return mass_1, mass_0

    def update_masses(self, t, y_true):
        """
        Updates the experts' masses based on their predictions.
        Experts with zero mass are removed, and the remaining masses are normalized.
        Returns True if only one expert remains, otherwise returns False.
        """

        # Penalize experts who made an incorrect prediction
        for i in list(self.experts_info.keys()):
            if self.experts_info[i]['predictions'][t] != y_true:
                self.experts_info[i]['mass'] *= self.beta  # Update the mass in experts info dictionary

        # Compute the total mass
        N_t = sum(info['mass'] for info in self.experts_info.values())

        print(f"Updated experts' masses: {[info['mass'] for info in self.experts_info.values()]}")
        print(f"N_t: {N_t}")

        # Normalize the masses
        for i in self.experts_info:
            self.experts_info[i]['mass'] /=  N_t

        print(f"Normalized experts' masses: {[info['mass'] for info in self.experts_info.values()]}")

        # Check the total mass
        total_mass = sum(info['mass'] for info in self.experts_info.values())
        print(f"Total mass after update: {total_mass}")

        # Remove experts with zero mass
        self.experts_info = {i: info for i, info in self.experts_info.items() if info['mass'] > 0}

        # Check if only one expert remains
        if len(self.experts_info) == 1:
            print("Only one expert remaining. Stopping the algorithm.")
            return True

        return False

    def run(self, X_test):
        """
        Runs the algorithm on the test data.
        """

        t = 0

        while t < len(X_test):

            print(f"\nRound: {t + 1}")

            # Collect predictions from each expert
            for i in self.experts_info:
                prediction = self.experts_info[i]['model'].predict(X_test[t].reshape(1, -1))[0]
                self.experts_info[i]['predictions'].append(prediction)

            predictions_list = [self.experts_info[i]['predictions'][t] for i in self.experts_info]
            print(f"Expert predictions for round {t + 1}: {predictions_list}")

            # Check if all the predictions are the same
            all_equal = all(pred == predictions_list[0] for pred in predictions_list)

            # Generate the adversarial label
            y_true = 1 - (np.bincount(predictions_list).argmax())
            print(f"Adversarial true label for round {t + 1}: {y_true}")

            # Coarsening
            print("\nCoarsening:")
            mass_1, mass_0 = self.coarsening(t)

            # Determine aggregated prediction
            aggregated_prediction = 1 if mass_1 >= mass_0 else 0
            print(f"Aggregated prediction: {aggregated_prediction}")

            if aggregated_prediction != y_true:
              print("The aggregated prediction is different from the true label.")
              self.total_errors += 1
              print(f"Update total errors: {self.total_errors}")

              # If experts agree completely, is not informative -> skip mass updating
              if all_equal:
                print("All experts agree completely. Skipping mass updating.")
                t += 1
                continue
              else:
                # If Errors occurs, update the masses
                print("\nMass updating:")
                stop = self.update_masses(t, y_true)
                if stop:
                  break
            else:
              print("The aggregated prediction is the same as the true label. No update is needed.")

            t += 1

        ########################################################################
        print()
        print("\nFinal Results:")
        print(f"\nTotal number of instances: {len(X_test)}")
        print(f"Remaining experts: {self.experts_info}")
        print(f"Total number of errors: {self.total_errors}")

In [3]:
# Test the algorithm
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_experts = 3
beta = 0.50

ewmwn = EWMWN(num_experts=num_experts, beta=beta)
ewmwn.initialize_experts(X_train, y_train)
ewmwn.run(X_test)

Initialized 3 experts: {0: {'model': SGDClassifier(), 'mass': 0.3333333333333333, 'predictions': []}, 1: {'model': SGDClassifier(), 'mass': 0.3333333333333333, 'predictions': []}, 2: {'model': SGDClassifier(), 'mass': 0.3333333333333333, 'predictions': []}}

Round: 1
Expert predictions for round 1: [0, 1, 0]
Adversarial true label for round 1: 1

Coarsening:
Experts predicting 1: {1}
Experts predicting 0: {0, 2}
Mass_1: 0.3333333333333333, Mass_0: 0.6666666666666666
Aggregated prediction: 0
The aggregated prediction is different from the true label.
Update total errors: 1

Mass updating:
Updated experts' masses: [0.16666666666666666, 0.3333333333333333, 0.16666666666666666]
N_t: 0.6666666666666666
Normalized experts' masses: [0.25, 0.5, 0.25]
Total mass after update: 1.0

Round: 2
Expert predictions for round 2: [0, 0, 0]
Adversarial true label for round 2: 1

Coarsening:
Experts predicting 1: set()
Experts predicting 0: {0, 1, 2}
Mass_1: 0.0, Mass_0: 1.0
Aggregated prediction: 0
The a